In [53]:
import json, random, re, collections, itertools, base64, sys
from dataclasses import dataclass
from copy import deepcopy
from pathlib import Path
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from operator import itemgetter as at
from IPython.core.display import display, HTML
from ipywidgets import interact
display_html = lambda x: display(HTML(str(x)))
from pprint import pprint as pp
from edit_distance import edit_distance
ls = lambda path: list(map(str, path.iterdir()))

sys.path.append('../src')
annot_path = Path("../annotations")
data_path = Path("../data")
preprocessed_path = Path("../preprocessed")
mturk_path = Path("../mturk/results")

def read_annotation(annotation_id, mturk_batch=None):
    if mturk_batch:
        annotation = annotation_io.mturk_annotation(mturk_batch, annotation_id)
    else:
        annotation = annotation_io.get_annotation(annotation_id)
    return annotation

import annotation_io
import read_data

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read mapping

In [46]:
with (preprocessed_path / "ingredients.json").open("r") as f:
    ingredients = json.load(f)
with (preprocessed_path / "labels.json").open("r") as f:
    idx2label = [tuple(t) for t in json.load(f)]
    label2idx = {r: i for i, r in enumerate(idx2label)}
with (preprocessed_path / "resources.json").open("r") as f:
    resources = json.load(f)
ingredient_dict = {k: v for k, v in ingredients}
resource_dict = {k: v for k, v in resources}

In [6]:
ls(preprocessed_path)

['../preprocessed/103308.npz',
 '../preprocessed/ingredients.json',
 '../preprocessed/resources.json',
 '../preprocessed/labels.json',
 '../preprocessed/103308.json']

In [7]:
states = np.load(str(preprocessed_path/ "103308.npz" ))
states.files

['103308_0',
 '103308_1',
 '103308_2',
 '103308_3',
 '103308_4',
 '103308_5',
 '103308_6',
 '103308_7',
 '103308_8',
 '103308_9']

In [8]:
states['103308_3']

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [34]:
from instruction_parsing import program, program_step, Instruction, execute

def human_program(annotation):
    """Runs the program and translates to human language"""
    actions = [
        (
            a.ts,
            a.command.name,
            ingredient_dict.get(a.arg, resource_dict.get(a.arg, "")),
            resource_dict.get(a.resource),
        )
        for a in program(annotation)
    ]
    return actions

In [52]:
mturk_batch=4286064
annotation_id="35L9RVQFCUDMD74V0GR1EN7XINTUHO"
annotation = read_annotation(annotation_id, mturk_batch)


pp (human_program(annotation))

NameError: name 'read_annotation' is not defined

In [47]:
with (mturk_path / f"{mturk_batch}.json").open('r') as f:
    annotations = json.load(f)
annotations

{'9068': ['320DUZ38GDHZZVYQ4B55MLMLM7IJGI',
  '34S9DKFK79KV4CABDFJ3MSA60NGNYR',
  '3AUQQEL7UBOBFMNZ7YUL4HAZAD10VI',
  '3LWJHTCVCIHA7I68OHJW11YVZN5FQ1'],
 '25531': ['35USIKEBNXBTLS057WOB31QD1D76NU',
  '38JBBYETQU5BCZJEV9SDGGCQRU5E4X',
  '3A1COHJ8NPQOFAXIXDQWIVP8LW1H8V',
  '3E4GGUZ1TEM4VLW1NHKSLWSXRD32K7',
  '3NJM2BJS421I4ULJX2E6TQNIXETPCB'],
 '45118': ['35L9RVQFCUDMD74V0GR1EN7XINTUHO',
  '3BWI6RSP7M48VM1XX7QKJSW0IW3E7P',
  '3JV9LGBJWZ9D06PNKOXAUH37L1PGOV',
  '3XLBSAQ9ZA766HSTTU4X42XI6207ZC'],
 '87264': ['3BXQMRHWK5T85L56RD7E6SFTZ9SMUP',
  '3NPFYT4IZIZ0UF3FLFK6DUU0X4QXGB',
  '3RXCAC0YIXKAFE27D78EXVD76V8G8D',
  '3RXPCZQMQV6S4EIMEMZZILQZSPI1GU'],
 '128355': ['3A4TN5196QDQRDNVEP61KHLRBNMHC1',
  '3KYQYYSHY12AOMFWIYW2OW0A892DOB',
  '3LOZAJ85YJ8AFLV8M0ZHBQ0ZNMS2X6',
  '3MYYFCXHJ929WDF514I1GJX249CG46'],
 '140729': ['3DEL4X4ELCG0G6OFFL6CKU1421MYXI',
  '3FFJ6VRIL7J6TG3Z8RV2PSK52B50IL',
  '3JJVG1YBEHSV1FBUZN9XK0CJOTRB53',
  '3K4J6M3CXKOOY2VMA5SSTMJ2WKLGAW',
  '3QECW5O0KNWVX15BZ90IKUI5UZC5T6'],
 '2

In [42]:
def annotator_agreement(annotation1,annotation2):
    lst1 = [(x.command.name,x.arg,x.resource) for x in program(annotation1)]
    lst2 = [(x.command.name,x.arg,x.resource) for x in program(annotation2)]
    dst,_ = edit_distance(lst1, lst2)
    return 1-dst/(len(lst1)+len(lst2))


In [58]:
html = []
for recipe_id in annotations:
    html.append(f"<h2>{recipe_id}</h2>")
    for p1,p2 in itertools.combinations(annotations[recipe_id],2):
        sim = annotator_agreement(read_annotation(p1, mturk_batch),read_annotation(p2, mturk_batch))
        line = f'<a href="http://127.0.0.1:8080/annotate/{mturk_batch}/{p1}" target="_new">{p1}</a>'
        line += ' ({s:0.2f}) '.format(s=sim)
        line += f'<a href="http://127.0.0.1:8080/annotate/{mturk_batch}/{p2}" target="_new">{p2}</a>'
        html.append(line)
html = ("<br />".join(html))
display_html(html)

In [ ]:
for a,b in zip(annotations["103308"], annotations["103308"][1:]):
    last = str(max(map(int,a.keys())))
    last = set(filter(lambda t: t[1].startswith('I'),map(tuple, a[last])))
    first = set(filter(lambda t: t[1].startswith('I'),map(tuple, b['0'])))
    print (last-first)

In [ ]:
program_step(annotations["103308"][1])

In [ ]:
s=set()
s.add('a')
s.remove('a')

In [31]:
edit_similarity([1,2,4,5,6,7],[1,2])

0.5